In [ ]:
!git clone https://github.com/vincentblot28/mapie_llm

Cloning into 'mapie_llm'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 9), reused 24 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 2.25 MiB | 7.57 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
!pip install git+https://github.com/scikit-learn-contrib/MAPIE.git@v1


  Cloning https://github.com/scikit-learn-contrib/MAPIE.git (to revision v1) to /tmp/pip-req-build-67agh_xz
  Running command git clone --filter=blob:none --quiet https://github.com/scikit-learn-contrib/MAPIE.git /tmp/pip-req-build-67agh_xz
  Running command git checkout -b v1 --track origin/v1
  Switched to a new branch 'v1'
  Branch 'v1' set up to track remote branch 'v1' from 'origin'.
  Resolved https://github.com/scikit-learn-contrib/MAPIE.git to commit 85f58d378e67a2fac00d999825205a68d1ba2366
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.2 MB/s eta 0:00:00
  Created wheel for MAPIE: filename=mapie-1.0.0a1-py3-none-any.whl size=185292 sha256=9ad9629a53f4c2d93b1e17b0b885970fdfbd006c6d122f330001f1cb5b3a2923
  Stored in directory: /tmp/pip-ephem-wheel-cache-kcah7dl6/wheels/d1/0a/cb/49dc0b75bb1cf582ba37b2b4c099c6e423d3a58e028751a649
Succe

In [ ]:
import json
import sys
sys.path.append('/content/mapie_llm')

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import torch
import torch.nn.functional as F
from mapie_v1.classification import SplitConformalClassifier
from mapie.metrics import classification_coverage_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils.prompt_utils import TransformCosmosQA

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
N_OBS = 1000
CONFIDENCE_LEVEL = 0.95

# Load data and transform it to X, y (X being a prompt)

In [ ]:
data = json.load(open("/content/mapie_llm/cosmosqa_10k.json"))

In [ ]:
data_transformer = TransformCosmosQA(data)
X, y = data_transformer.transform_data()

# Load the model and its tokenizer

In [ ]:
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    model.eval()
    return tokenizer, model

# %%
tok, mo = load_model("mistralai/Mistral-7B-Instruct-v0.3")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Wrap the model

In [ ]:
print(X[0])

Context: It 's got character . You know what they say : ' If it ai n't too broke , do n't fix it ' . " Howard stood up in his pyjamas that smelled of Vince , and looked out of the window . He repeated Vince 's aphorism over and over until his heartbeat settled a little .
Question: What was Howard doing when he walked to the window
Choices:
A. He was sneezing
B. He was chewing gum
C. He was scratching his lap
D. He was thinking
E. I don't know
F. None of the above
Answer: D
Context: My house is constantly getting messy and I ca n't keep up . I am starting at a new school with no one I know and it is 4 times bigger than UAF . I am now going to have to balance school , homework , kids , bill paying , appointment making and cleaning when I can barely keep up without the school and homework ( keep in mind this is a full time GRADUATE program at a fairly prestigious school ) . We are in financial crisis .
Question: What is causing the narrator 's recent stress ?
Choices:
A. They are moving t

In [ ]:
class LLMClassifier:


    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.options = ["Answer: A", "Answer: B", "Answer: C", "Answer: D"]
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.option_ids = [self.tokenizer.encode(opt)[-1] for opt in self.options]
        self.is_fitted_ = True
        self.classes_ = [0, 1, 2, 3]

    def fit(self):
      pass


    def predict_proba(self, prompts):
        if isinstance(prompts, str):
            prompts = [prompts]  # Convert single prompt to list
        pred_probas = []
        for prompt in tqdm(prompts):
            prompt = [prompt]
            inputs = self._prepare_inputs(prompt)
            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits[:, -1, :]  # Extract logits for the last token [batch_size, vocab_size]
            logits_options = logits[:, self.option_ids]  # Select logits corresponding to answer options

            probabilities = F.softmax(logits_options, dim=-1).cpu().numpy().flatten()
            probabilities /= probabilities.sum() # Ensure that the sum is exaclty equal to 1, which may not be true in some cases
            pred_probas.append(probabilities)
        return np.array(pred_probas)

    def predict(self, prompts):
        probas = self.predict_proba(prompts)
        return np.argmax(probas, axis=-1)

    def _prepare_inputs(self, prompts):
        inputs = self.tokenizer(prompts, return_tensors="pt", truncation=True)
        for k in inputs:
            if torch.is_tensor(inputs[k]):
                inputs[k] = inputs[k].to("cuda")
        return inputs

In [ ]:
llm = LLMClassifier(tok, mo)

In [ ]:
scc = SplitConformalClassifier(estimator=llm, confidence_level=CONFIDENCE_LEVEL)

In [ ]:
X_small, y_small = X[:N_OBS], y[:N_OBS]

In [ ]:
X_conf, X_test, y_conf, y_test = train_test_split(X_small, y_small, train_size=.5)

In [ ]:
scc.conformalize(X_conf, y_conf)

100%|██████████| 500/500 [08:45<00:00,  1.05s/it]


In [ ]:
y_pred, y_pred_set = scc.predict_set(X_test)

In [ ]:
classification_coverage_score(y_test, y_pred_set[:, :, 0])

In [ ]:
def print_examples_and_prediction_set(data_transformer, X, y, y_pred, y_pred_set):
  sizes = y_pred_set.sum(axis=1)
  unique_sizes= np.unique(sizes)

  for size in unique_sizes:
    all_idx_size = np.argwhere(sizes==size)[:, 0]
    random_idx = np.random.choice(all_idx_size, 1)[0]

    prompt_without_few_shot = X[random_idx].split("Answer:")[-2][3:]
    label = y[random_idx]
    pred = y_pred[random_idx]
    pred_set = np.argwhere(y_pred_set[random_idx][:, 0])[0]
    print("==========================================",
        prompt_without_few_shot,
        f"Correct answer: {data_transformer.label_encoder.inverse_transform([label])[0]}",
        f"Prediction: {data_transformer.label_encoder.inverse_transform([pred])[0]}",
        f"Prediction set: {data_transformer.label_encoder.inverse_transform(pred_set)}",
        sep="\n")

In [ ]:
print_examples_and_prediction_set(data_transformer, X_test, y_test, y_pred, y_pred_set)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)


In [ ]:
print(f"The LMM is right {round(accuracy * 100, 2)} % of the time. \n")


In [ ]:
def plot_label_distribution(y_pred_set):
    num_labels_per_sample = np.sum(y_pred_set, axis=1)
    label_counts = np.bincount(num_labels_per_sample)

    min_number = np.min(num_labels_per_sample)
    max_number = np.max(num_labels_per_sample)

    fig = go.Figure()

    fig.add_trace(go.Bar(x=list(range(min_number, max_number + 1)), y=label_counts[min_number:max_number + 1]))

    fig.update_layout(
        title='Distribution of the size of prediction sets',
        xaxis_title='Size',
        yaxis_title='Number of prediction sets',
        xaxis=dict(tickmode='linear', tick0=min_number, dtick=1),
        yaxis=dict(tickmode='linear', tick0=0, dtick=5)
    )

    fig.show()

In [ ]:
plot_label_distribution(y_pred_set[:, :, 0])